In [1]:
!git clone https://github.com/ntluongg/QA_Vietnamese.git

Cloning into 'QA_Vietnamese'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 168 (delta 62), reused 124 (delta 33), pack-reused 0
Receiving objects: 100% (168/168), 19.52 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [2]:
!pip install gdown

In [3]:
from transformers import BertTokenizer, BertConfig
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import gdown

url = 'https://drive.google.com/drive/folders/19kntGT7S5kAU2yGfC3duCesg0F59ic52?usp=share_link'
gdown.download_folder(url, quiet=True, use_cookies = False,)

['/kaggle/working/pretrained_bertQA_mid/config.json',
 '/kaggle/working/pretrained_bertQA_mid/model.safetensors',
 '/kaggle/working/pretrained_bertQA_mid/special_tokens_map.json',
 '/kaggle/working/pretrained_bertQA_mid/tokenizer_config.json',
 '/kaggle/working/pretrained_bertQA_mid/training_args.bin',
 '/kaggle/working/pretrained_bertQA_mid/vocab.txt']

In [5]:
from QA_Vietnamese.train.model.bert.BERT_model import QaBERT
from QA_Vietnamese.train.model.bert.train_bert import evaluate
from QA_Vietnamese.train.model.bert.dataloader import load_squad_to_torch_dataset

In [6]:
class Args:
    do_lower_case = True
    folder_model = "/kaggle/working/pretrained_bertQA_mid"

    path_input_test_data = "/kaggle/working/QA_Vietnamese/train/dataset/data/val_origin_1k.csv"

    no_cuda = False
    n_gpu = 1
    device = "cuda:0"
    seed = 42

    max_seq_length = 400
    max_query_length = 64
    weight_class = [1, 1]


args = Args()

device = torch.device(args.device)
tokenizer = BertTokenizer.from_pretrained(args.folder_model, do_lower_case=args.do_lower_case)

config = BertConfig.from_pretrained(args.folder_model)

# # custom some parameter for custom bert
config = config.to_dict()
config.update({"device": args.device})
config = BertConfig.from_dict(config)

model = QaBERT.from_pretrained(args.folder_model, config=config)

model = model.to(device)

In [7]:

path_input_data = args.path_input_test_data
#change to your desired folder
path_input_data_pt = "/kaggle/working/test_data.pt"


eval_dataset, eval_dataloader = load_squad_to_torch_dataset(path_input_data,
                                                            tokenizer,
                                                            args.max_seq_length,
                                                            args.max_query_length,
                                                            12,
                                                            is_training=True)
torch.save(eval_dataset, path_input_data_pt)
    
# Eval!
print("***** Running evaluation")
print("  Num examples = %d", len(eval_dataset))

total_loss = 0.0
l_full_predict = []
l_full_target = []

for batch in tqdm(eval_dataloader, desc="Evaluating", leave=False):
    model.eval()
    batch = tuple(t.to(args.device) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'label': batch[3]
                  }

        loss, l_predict, l_target = model.compute_loss(inputs['input_ids'],
                                inputs['attention_mask'],
                                inputs['token_type_ids'],
                                inputs['label'])
        total_loss += loss.item()
        l_full_predict.extend(l_predict)
        l_full_target.extend(l_target)


        
f1_score_micro = f1_score(l_full_target, l_full_predict)
accuracy = accuracy_score(l_full_target, l_full_predict)
precision = precision_score(l_full_target, l_full_predict)
recall = recall_score(l_full_target, l_full_predict)

output_validation = {
    "loss": round(total_loss / len(eval_dataset), 3),
    "accuracy": round(accuracy, 3),
    "f1": round(f1_score_micro, 3),
    "precision": round(precision, 3),
    "recall": round(recall, 3)
}

print(output_validation)

***** Running evaluation
  Num examples = %d 1114


/tmp/ipykernel_42/3314380529.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm(eval_dataloader, desc="Evaluating", leave=False):


Evaluating:   0%|          | 0/93 [00:00<?, ?it/s]

{'loss': 0.045, 'accuracy': 0.903, 'f1': 0.846, 'precision': 0.851, 'recall': 0.841}
